## TFLite inference

Test TensorFlow Lite inference on an input image.

In [ ]:
import os
import cv2
import numpy as np
from os.path import join
import tensorflow as tf


import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Load TFLite model and allocate tensors.
model_path="../data/tflite_models/converted_model_224_3.tflite"

interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
print(output_details)

interpreter.allocate_tensors()

### Test on an image

In [ ]:
INPUT_SIZE = (224, 224)

test_img = cv2.imread("../data/test_examples/test_img.png")
test_img = test_img[:,:,::-1]
test_img = cv2.resize(test_img, INPUT_SIZE)

# Test model on the input image.
input_shape = input_details[0]['shape']
input_data = np.array((test_img.astype(np.float32) / 255.0)[np.newaxis, :, :, :])
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data.shape)


### Profile execution

In [ ]:
import time

n = 100
start_time = time.time()
for _ in range(n):
    interpreter.invoke()

print("FPS:", round(n/(time.time() - start_time), 2))

### Visualize predictions

In [ ]:
threshold = 100

out_img = np.squeeze(output_data)
plt.imshow(test_img)
plt.imshow(out_img*255 > threshold, alpha=0.4)
plt.show()